#### Data Cleaning Function

In [ ]:
def text_processor_em(documents):
    stopWords = set(nltk.corpus.stopwords.words('english'))

    articles = []
    word_id = {}
    id_word = {}
    current_wordid = 0
    wordID = []
    Countword = []
    singlecount = [] 
    store1 = []
    store2 = []
    for doc in documents:
        word_count = {}
            ## split words
        word_split = doc.split()
        for word in word_split:
            word = word.lower().strip()
            ##
            if (len(word)) > 1 and  word.isalpha() and word not in stopWords:
                if word not in word_id:
                    word_id[word] = current_wordid
                    id_word[current_wordid] = word
                    current_wordid += 1
                if word in word_count:
                    word_count[word] += 1
                else:
                    word_count[word] = 1
                
        wordIDList = []
        wordCountList = []
        wordCount = 0

        for word in word_count.keys():
            wordIDList.append(word_id[word])
            wordCountList.append(word_count[word])
            wordCount = wordCount + word_count[word] #total words in a doc
        wordID.append(wordIDList)
        Countword.append(wordCount)
        singlecount.append(wordCountList)
        store1.append(wordIDList)
        store1.append(wordCountList)
        store1.append(wordCount)    
        store1 = []
        store2.append(store1)
    return wordID,Countword,singlecount,word_id,id_worda

#### Initialization and EM algorithm

In [1]:
def maxwordnum():
    length = []
    for l in wordID:
        length.append(len(l))
    return max(length)   

def initial():
    for z in range(0, K):
        for w in range(0, N):
            ntw[z, w] += 1.0/N + np.random.random() # probabily of word under every topic
            nt[z] += ntw[z, w] 
    updatebeta(beta,K,N, ntw, nt)

In [ ]:
%%cython

import cython
cimport numpy as np
import numpy as np
from libc.math cimport log
cimport scipy.special.cython_special
from scipy.special.cython_special import psi


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

def updatebeta(double[:,:] beta, int K,int N, double[:,:] ntw, double[:] nt):
    
    cdef int z,w
    for z in range(0,K):
        for w in range(0,N):
            if(ntw[z,w] > 0):
                beta[z,w] = beta[z,w] = log(ntw[z,w] / nt[z])
            else:
                beta[z,w] = -100

In [ ]:
%%cython

import cython
cimport numpy as np
import numpy as np
from libc.math cimport exp,log
cimport scipy.special.cython_special
from scipy.special.cython_special import psi


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

def variational(double[:,:] gamma, double[:,:] phi, singlecount,wordID, Countword, double[:,:] beta,
                int alpha, int d, int K, int infer_iter):
    """
    Variational with E step
    
    Parameters
    ----------
    gamma         : a matrix, shape(M,K), latent parameter of a distribution of topic for each document
    phi           : a matrix, shape(K,v), latent parameter of a distribution of words for each topic
    wordID        : a list returned by text_processor method
    Countword     : a list returned by text_processor method
    beta          : a matrix, returned by initial method
    alpha         : a fix number 
    d             : a fix number 
    K             : number of topics, a fix number
    infer_iter    : number of iteration
    
    Returns
    -------gamma
    gamma         : updated gamma, a matrix, shape(M,K), distribution of topic for each document
   
   """

    cdef int wordlength = len(wordID[d])
    cdef double totalphi = 0
    cdef double[:] phi_old = np.zeros((K))
    cdef double [:] gamma_prime = np.zeros((K))
    cdef int z, iteration,w

    # initialize parameter
    for z in range(K):
        #initialize gamma
        gamma[d][z] = alpha + Countword[d] * 1.0 / K
        gamma_prime[z] = psi(gamma[d][z])
        # initialize phi
        for w in range(0,len(wordID[d])):
            phi[w,z] = 1.0 / K
    
    for iteration in range(infer_iter):
        for w in range(wordlength):
            totalphi = 0
            for z in range(0,K):
                phi_old[z] = phi[w,z]
                phi[w,z] = beta[z,wordID[d][w]] + gamma_prime[z]
                if z > 0:
                    totalphi = log((exp(totalphi)) + exp(phi[w,z]))
                else:
                    totalphi = phi[w,z]
            for z in range(0,K):
                phi[w,z] = exp(phi[w,z] - totalphi)
                gamma[d][z] = gamma[d][z] + singlecount[d][w] * (phi[w,z] - phi_old[z])
                gamma_prime[z] = psi(gamma[d][z]) 
                
    return gamma

In [ ]:
perp = []
for iteration in range(0,EM_iter):
    nt = np.zeros((K))
    ntw = np.zeros((K,N))
    alphass = 0
    
        # E step
    for d in range(0,M):
        variational(gamma, phi, singlecount,wordID, Countword, beta, alpha, d, K, infer_iter)
        totalgamma = 0
        for z in range(0,K):
            totalgamma += gamma[d,z]
            alphass += psi(gamma[d,z])
        alphass -= K * psi(totalgamma)
        
        for w in range(len(wordID[d])):
            for z in range(0,K):
                ntw[z][wordID[d][w]] += singlecount[d][w] * phi[w,z]
                nt[z] += singlecount[d][w] * phi[w,z]
    # M step
    updatebeta(beta,K,N, ntw, nt)
    
    # perplexity
    nd = np.sum(np.array(gamma),1)
    n = 0
    ll = 0.0

    for d,doc in enumerate(wordID):
        for w in doc:
            ll = ll + np.log(((np.array(ntw)[:, w] / np.array(nt))* (np.array(gamma)[d, :] / nd[d])).sum())
            n = n + 1
            exp_ll = np.exp(ll/(-n))
    perp.append(exp_ll) 